In [141]:
!pip install -q translators sacrebleu

In [142]:
import pandas as pd
import translators as ts
import sacrebleu

In [23]:
df_dev_tcn = pd.read_csv('../4-title-translation/product-translation-dataset/dev_tcn.csv')
df_dev_en = pd.read_csv('../4-title-translation/product-translation-dataset/dev_en.csv')

In [29]:
df_dev_tcn.tail()

,text,split
995,【HippoRed】撕破乐趣★独特风格★中直筒牛仔裤 O591_445,private
996,兒童套裝 台灣製薄長袖居家套裝 魔法Baby~k60092,private
997,LONGCHAMP Le Pliage Neo高密尼龍後背包(中型),private
998,iFairies 開口可調節戒指★ifairies【56472】【56472】,private
999,PolarStar 女 排汗快干T恤『黑』P18102,private


In [144]:
df_dev_en.tail()

,translation_output
995,Hippored Torn Fun Unique Style Straight Jeans ...
996,Kids Set Table Bay - Thin Long Sleeve Home Sui...
997,LONGCHAMP Le Pliage Neo High Density Nylon Bac...
998,IFairies Opening Adjustable Ring ifairies [564...
999,PolarStar Women Sweat Quick Dry T-shirt Black ...


In [118]:
df_dev_tcn['text_clean'] = df_dev_tcn['text'].apply(lambda x : x.replace('.', ' '))

In [119]:
prep = '. '.join(df_dev_tcn['text_clean'])

In [120]:
len(prep)

31662

In [121]:
len(prep.split('. '))

1000

In [154]:
def translate_batch(text):
    res = ts.google(text,
                     use_domain='general',
                     from_language='zh',
                     sleep_seconds=0.4)
    return res.split('. ')

In [164]:
INTERVAL = 1000
results = []
now_str = ''
for i in range(len(df_dev_tcn)):
    if len(now_str) + len(df_dev_tcn['text_clean'][i]) <= INTERVAL:
        now_str += '. ' + df_dev_tcn['text_clean'][i] 
    else:
        print("translating 'till: {}".format(i))
        results.extend(translate_batch(now_str))
        now_str = ''
if len(now_str) > 0:
    print("translating 'till end")
    results.extend(translate_batch(now_str))

translating 'till: 34
[21:24:19] UseTimeSeconds(fn: google_api): 0.89
translating 'till: 65
[21:24:20] UseTimeSeconds(fn: google_api): 0.96
translating 'till: 100
[21:24:21] UseTimeSeconds(fn: google_api): 0.86
translating 'till: 130
[21:24:22] UseTimeSeconds(fn: google_api): 0.84
translating 'till: 164
[21:24:23] UseTimeSeconds(fn: google_api): 1.0
translating 'till: 195
[21:24:24] UseTimeSeconds(fn: google_api): 0.98
translating 'till: 230
[21:24:25] UseTimeSeconds(fn: google_api): 1.01
translating 'till: 262
[21:24:25] UseTimeSeconds(fn: google_api): 0.84
translating 'till: 291
[21:24:27] UseTimeSeconds(fn: google_api): 1.31
translating 'till: 322
[21:24:28] UseTimeSeconds(fn: google_api): 1.17
translating 'till: 355
[21:24:29] UseTimeSeconds(fn: google_api): 0.86
translating 'till: 387
[21:24:30] UseTimeSeconds(fn: google_api): 0.95
translating 'till: 419
[21:24:31] UseTimeSeconds(fn: google_api): 0.92
translating 'till: 450
[21:24:32] UseTimeSeconds(fn: google_api): 0.97
translati

In [165]:
len(results)

1004

In [166]:
results[:5]

['',
 'OPPO A75 A75s A73 mobile phone shell soft shell lanyard shell big eye rabbit silicone shell',
 'SOFT 99 coated car wax (strong water repellent type)',
 'Low-sugar dried mango 250g Zhenyuxing',
 '＊Trail culture ＊ROUND TOP space craft imported from Japan- diamond (SC-MK-010)']

In [167]:
results[-5:]

['[HippoRed] fun to tear ★ unique style ★ straight jeans O591_445',
 "Children's suit Taiwan-made thin long-sleeved home suit magic Baby~k60092",
 'LONGCHAMP Le Pliage Neo high-density nylon backpack (medium size)',
 '',
 'PolarStar Women\'s Perspiration Quick-drying T-shirt "Black" P18102']

In [170]:
df_dev_tcn['text'].values[-5:]

array(['【HippoRed】撕破乐趣★独特风格★中直筒牛仔裤 O591_445',
       '兒童套裝 台灣製薄長袖居家套裝 魔法Baby~k60092',
       'LONGCHAMP Le Pliage Neo高密尼龍後背包(中型)',
       'iFairies 開口可調節戒指★ifairies【56472】【56472】',
       'PolarStar 女 排汗快干T恤『黑』P18102'], dtype=object)

In [169]:
df_dev_en['translation_output'].values[-5:]

array(['Hippored Torn Fun Unique Style Straight Jeans O591 445',
       'Kids Set Table Bay - Thin Long Sleeve Home Suit Magic Baby ~ K60092',
       'LONGCHAMP Le Pliage Neo High Density Nylon Backpack (medium)',
       'IFairies Opening Adjustable Ring ifairies [56472] 【 56472 】',
       'PolarStar Women Sweat Quick Dry T-shirt Black 』 P18102PolarStar'],
      dtype=object)

In [173]:
sacrebleu.corpus_bleu(['IFairies Opening Adjustable Ring ifairies [56472] 【 56472 】'],
                      [['IFairies Opening Adjustable Ring ifairies [56472] 【 56472 】']],
                      lowercase=True,
                      tokenize='13a',
                      use_effective_order= False).score

100.00000000000004

In [176]:
sacrebleu.corpus_bleu(['iFairies Opening Adjustable Ring ifairies 【56472】 【 56472 】'],
                      [['IFairies Opening Adjustable Ring ifairies [56472] 【 56472 】']],
                      lowercase=True,
                      tokenize='13a',
                      use_effective_order= False).score

47.79995354275012

In [139]:
results_ = results[1:]

In [163]:
sacrebleu.corpus_bleu(results_,
                      [df_dev_en['translation_output']],
                      lowercase=True,
                      tokenize='13a',
                      use_effective_order= False).score

10.34014305798428

In [162]:
sacrebleu.corpus_bleu(df_dev_en['translation_output'],
                      [df_dev_en['translation_output']],
                      lowercase=True,
                      tokenize='13a',
                      use_effective_order= False).score

100.00000000000004

In [36]:
df_dev_tcn['translation_output'] = ts.alibaba(list(df_dev_tcn['text'].values),
                                              use_domain='general',
                                              from_language='zh',
                                              sleep_seconds=0.4)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [31]:
df_dev_tcn.tail()

,text,split,translation_output
995,【HippoRed】撕破乐趣★独特风格★中直筒牛仔裤 O591_445,private,0 phone case soft case lanyard shell big eye r...
996,兒童套裝 台灣製薄長袖居家套裝 魔法Baby~k60092,private,0 phone case soft case lanyard shell big eye r...
997,LONGCHAMP Le Pliage Neo高密尼龍後背包(中型),private,0 phone case soft case lanyard shell big eye r...
998,iFairies 開口可調節戒指★ifairies【56472】【56472】,private,0 phone case soft case lanyard shell big eye r...
999,PolarStar 女 排汗快干T恤『黑』P18102,private,0 phone case soft case lanyard shell big eye r...


In [7]:
wyw_text = '季姬寂，集鸡，鸡即棘鸡。棘鸡饥叽，季姬及箕稷济鸡。'
chs_text = '季姬感到寂寞，罗集了一些鸡来养，鸡是那种出自荆棘丛中的野鸡。野鸡饿了唧唧叫，季姬就拿竹箕中的谷物喂鸡。'

In [17]:
print(ts.deepl(wyw_text, from_language='zh'))

[20:38:43] UseTimeSeconds(fn: deepl_api): 3.58
Ji Ji is silent, and collects chickens, the chickens are the spiny chickens.If the fowls are hungry, Jiji and Jiji will help the fowls.


In [18]:
print(ts.alibaba(wyw_text, use_domain='general', from_language='zh'))

[20:38:47] UseTimeSeconds(fn: alibaba_api): 2.49
Ji Ji is quiet, collecting chicken, chicken is chicken. Braised Chicken, Ji Ji and Ji ji ji chicken.


In [28]:
print(ts.alibaba([wyw_text, chs_text]*4, use_domain='general', from_language='zh', sleep_seconds=0.4))

[20:45:58] UseTimeSeconds(fn: alibaba_api): 2.1
['Ji Ji is quiet, chicken, chicken is chicken. Braised Chicken, Ji Ji and Ji ji ji chicken. ', 'Ji Ji felt lonely and gathered some chickens to raise. Chickens are pheasants from thorns. When pheasants are hungry, Ji Ji will feed the chicken with the grain in the bamboo dustpan. ',' Ji Ji Ji, collect chicken, chicken is chicken. Braised Chicken, Ji Ji and Ji ji ji chicken. ', 'Ji Ji felt lonely and gathered some chickens to raise. Chickens are pheasants from thorns. When pheasants are hungry, Ji Ji will feed the chicken with the grain in the bamboo dustpan. ',' Ji Ji Ji, collect chicken, chicken is chicken. Braised Chicken, Ji Ji and Ji ji ji chicken. ', 'Ji Ji felt lonely and gathered some chickens to raise. Chickens are pheasants from thorns. When pheasants are hungry, Ji Ji will feed the chicken with the grain in the bamboo dustpan. ',' Ji Ji Ji, collect chicken, chicken is chicken. Braised Chicken, Ji Ji and Ji ji ji chicken. ', 'Ji J

In [26]:
?ts.alibaba

Signature: ts.alibaba(query_text, from_language='auto', to_language='en', **kwargs)
Docstring:
https://translate.alibaba.com
:param query_text: string, must.
:param from_language: string, default 'auto'.
:param to_language: string, default 'en'.
:param **kwargs:
        :param use_domain: string, default 'message', choose from ("general","message","offer")
        :param is_detail_result: boolean, default False.
        :param proxies: dict, default None.
        :param sleep_seconds: float, >0.05. Best to set it yourself, otherwise there will be surprises.
:return: string or dict
File:      c:\users\franz\appdata\local\programs\python\python36\lib\site-packages\translators\apis.py
Type:      method


In [19]:
print(ts.google(wyw_text, if_use_cn_host=True, from_language='zh'))

[20:38:48] UseTimeSeconds(fn: google_api): 0.82
Ji Jiji, collect chickens, chickens are spiny chickens. The spiny chicken is hungry, Ji Ji and Ji Jiji chicken.


In [20]:
print(ts.bing(wyw_text, if_use_cn_host=False, from_language='zh'))

[20:38:51] UseTimeSeconds(fn: bing_api): 0.41
Ji Ji is silent, set chicken, chicken is the chicken. The ratchets are hungry, and the jiji and the roosters are in their sights.


In [21]:
?ts.bing

Signature: ts.bing(query_text, from_language='auto', to_language='en', **kwargs)
Docstring:
http://bing.com/Translator, http://cn.bing.com/Translator.
:param query_text: string, must.
:param from_language: string, default 'auto'.
:param to_language: string, default 'en'.
:param **kwargs:
        :param if_use_cn_host: boolean, default None.
        :param is_detail_result: boolean, default False.
        :param proxies: dict, default None.
        :param sleep_seconds: float, >0.05. Best to set it yourself, otherwise there will be surprises.
:return: string or list
File:      c:\users\franz\appdata\local\programs\python\python36\lib\site-packages\translators\apis.py
Type:      method
